# 예제 3.44-3.46: 검증용 데이터세트를 통한 평가

## 학습목표
1. **훈련/검증/테스트 데이터 분할** 방법 익히기
2. **random_split()** 함수 사용법 학습하기
3. **과적합(Overfitting) 방지**를 위한 검증 데이터 활용 이해하기
4. **모델 평가** 방법 학습하기

---

#### 예제 3.44 라이브러리 및 데이터 준비

In [ ]:
import torch
import pandas as pd
from torch import nn
from torch import optim
# random_split: 데이터셋을 무작위로 분할
from torch.utils.data import Dataset, DataLoader, random_split

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

---

#### 커스텀 데이터셋 및 모델 클래스

In [ ]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

---

#### 예제 3.45 데이터 분할

**데이터 분할 비율 (일반적)**
- 훈련(Train): 80% - 모델 학습용
- 검증(Validation): 10% - 하이퍼파라미터 튜닝용
- 테스트(Test): 10% - 최종 성능 평가용

In [ ]:
# 전체 데이터셋 생성
dataset = CustomDataset("../datasets/non_linear.csv")
dataset_size = len(dataset)

# 데이터 분할 크기 계산 (80:10:10 비율)
train_size = int(dataset_size * 0.8)       # 훈련 데이터: 80%
validation_size = int(dataset_size * 0.1)  # 검증 데이터: 10%
test_size = dataset_size - train_size - validation_size  # 테스트: 나머지

# random_split으로 데이터셋 분할
train_dataset, validation_dataset, test_dataset = random_split(
    dataset, [train_size, validation_size, test_size]
)
print(f"Training Data Size : {len(train_dataset)}")
print(f"Validation Data Size : {len(validation_dataset)}")
print(f"Testing Data Size : {len(test_dataset)}")

# 각 데이터셋에 대한 데이터로더 생성
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

In [ ]:
# 모델, 손실함수, 옵티마이저 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

---

#### 모델 학습

In [ ]:
# 훈련 데이터로 모델 학습
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

---

#### 예제 3.46 검증 데이터로 모델 평가

**검증의 목적**
- 훈련 중 모델의 일반화 성능 확인
- 과적합 여부 판단

In [ ]:
# 검증 데이터로 모델 평가
with torch.no_grad():  # 기울기 계산 비활성화
    model.eval()  # 평가 모드
    
    for x, y in validation_dataloader:
        x = x.to(device)
        y = y.to(device)
        
        outputs = model(x)
        print(f"X : {x}")           # 입력값
        print(f"Y : {y}")           # 실제값
        print(f"Outputs : {outputs}")  # 예측값
        print("--------------------")